# MATH2504 Project 2 2022 Semester 2 Submission

[Assignment Instructions](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/project2.html)

Student names: Limao Chang, Tiarne Graves

## Perspective Seminar

## Task 1

Features of amusement park that are not captured by model:
- Amusement park attractions usually serve multiple people simultaneously, whereas the model assumes jobs can only be serviced one at a time
- People may cut in line and skip queues, so in reality it is not really a queue in the first-in, first-out (FIFO) sense

Model assumptions that are unrealistic for the application of a theme park:
- Assuming that queues (and by extension the theme park) have infinite capacity
- Assuming that people will unconditionally stay in a queue until they get to the front of the line - they may leave because the queue is too long, they want to go to another attraction, etc.
- Assuming that the time taken to move between attractions or leave the theme park is instant, as travelling between attractions takes time
- Service duration is independent from all other services - if people are at the theme park together as a party (e.g. a family travelling around the theme park), their service durations will not be independent from each other
- Arrival of jobs to nodes is exponentially distributed

Answer:  If you were actually using this model to assess congestion at the park, would it be useful or not?


## Task 2

## Task 3

## Task 4

## Task 5